# Web Search Agent
This notebook demonstrates a web search agent using SwarmZero framework and Tavily API.

In [ ]:
import os

import agentops
from dotenv import load_dotenv
from swarmzero import Agent
from tavily import TavilyClient

load_dotenv()
agentops.init(os.getenv("AGENTOPS_API_KEY"))
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

## Define Search Functions

In [ ]:
async def web_search(query: str) -> dict:
    response = tavily_client.search(query)
    results = []
    for result in response["results"][:3]:
        results.append({"title": result["title"], "url": result["url"], "content": result["content"]})
    return results


async def extract_from_urls(urls: list[str]) -> dict:
    response = tavily_client.extract(urls=urls)

    if response["failed_results"]:
        print(f"Failed to extract from {response['failed_results']}")

    results = []
    for result in response["results"]:
        results.append({"url": result["url"], "raw_content": result["raw_content"]})

    return results

## Initialize and Run Agent

In [ ]:
my_agent = Agent(
    name="workflow-assistant",
    functions=[
        web_search,
        extract_from_urls,
    ],
    config_path="./swarmzero_config.toml", # see https://github.com/swarmzero/swarmzero/blob/main/swarmzero_config_example.toml
    instruction="You are a helpful assistant that can search the web and extract information from a given URL.",
    # chat_only_mode=True  # remove comment only if using `my_agent.chat()`
)

### Option 1: Chat directly without starting the agent's server

In [ ]:
import asyncio

In [ ]:
response = asyncio.run(my_agent.chat(prompt="what is Decentralized-AI about about?"))

In [ ]:
print(response)

### Option 2: Run agent server

In [ ]:
# my_agent.run()  # see agent API at localhost:8000/docs

Send a message to the agent:
```bash
curl -X 'POST' \
  'http://localhost:8000/api/v1/chat' \
  -H 'accept: application/json' \
  -H 'Content-Type: multipart/form-data' \
  -F 'user_id=test_user' \
  -F 'session_id=test_web_search_agent' \
  -F 'chat_data={"messages":[{"role":"user","content":"what is swarmzero.ai about?"}]}'
```